In [10]:
#инициализация графа доступа. 
#Где S_i - субъекты, некоторые из которых имеют доступ к данным других субъектов, 
#могут давать доступ, могут и то, и другое
def stare_initiala():
    graf = {'S1':[['S2','t'],['S3','t','g']],
            'S2':[['x','r','w']],
            'S3':[['x','r']],
            'S4':[['y','w'],['S2','t','g']],
            'S5':[['y','r','w'],['S2','g']],
            'S6':[['S1','t','g']],
            'S7':[['S3','g']],
            'S8':[['z','r'],['S1','t','g']],
            'S9':[['S8','t']],
            'S10':[['S4','t','g']]
    }
    return graf

#вывод графа на экран
def print_graf(graf):
    for item,val in graf.items():
        for value in val:
            print (item + " -", end = '')
            for i in range(1,len(value)):
                print (value[i], end = '')
            print ("-> " + value[0])

In [11]:
#поиск тех, кому дается доступ
def grants(x,graf):
    nodes = list()
    for item,val in graf.items():
        if item == x:
            for value in val:
                for i in range(1, len(value)):
                    if value[i] == 'g':
                        nodes.append(value[0])
    if len(nodes) == 0: return False
    else: return nodes

#поиск тех, у кого берется доступ
def takes(x,graf):
    nodes = list()
    for item,val in graf.items():
        if item == x:
            for value in val:
                for i in range(1, len(value)):
                    if value[i] == 't': nodes.append(value[0])
    if len(nodes)==0: return False
    else: return nodes

#поиск тех, кто берет доступ
def who_takes(x, graf):
    nodes = list()
    for item,val in graf.items():
        for value in val:
            if(value[0] == x):
                for i in range(1, len(value)):
                    if value[i] == 't': nodes.append(item)
    if len(nodes)==0: return False
    else: return nodes

#поиск тех, кто дает доступ
def who_grats(x,graf):
    nodes = list()
    for item,val in graf.items():
        for value in val:
            if(value[0] == x):
                for i in range(1, len(value)):
                    if value[i] == 'g': nodes.append(item)
    if len(nodes)==0: return False
    else: return nodes

In [12]:
#список тех, у кого берется доступ
def noduri_intind_terminal(x,graf,nodes):
    nodes_take = takes(x, graf)
    if(nodes_take != False):
        for nod in nodes_take:
            nodes.append(nod)
            noduri_intind_terminal(nod,graf,nodes)
    return nodes

#список тех, кто берет доступ
def noduri_intinde_terminal(y,graf,nodes):
    nodes_that_take = who_takes(y, graf)
    if(nodes_that_take != False):
        for nod in nodes_that_take:
            nodes.append(nod)
            noduri_intinde_terminal(nod,graf,nodes)
    return nodes

#список тех, кто дает доступ
def noduri_itinde_initial(y,graf):
    nodes = set()
    nodes_that_grant = who_grats(y,graf)
    if nodes_that_grant != False:
        for nod in nodes_that_grant:
            nodes.update(noduri_intinde_terminal(nod,graf,list()))
    nodes = list(nodes)
    if len(nodes) > 0: return nodes
    else: return False

In [15]:
#проверка на то, что введен субъект(большая буква)
def is_subject(x):
    if x.isupper(): return True
    else: return False

#является ли мостом 
def is_bridge(x,y,graf):
    if x == y: return True
    elif x in noduri_intinde_terminal(y,graf,list()): return True
    elif y in noduri_intinde_terminal(x,graf,list()): return True
    noduri = noduri_intind_terminal(y,graf,list())
    if noduri !=False:
        for nod in noduri:
            noduri = who_grats(nod,graf)
            if(noduri != False):
                for nod_grant in noduri:
                    if x in noduri_intinde_terminal(nod_grant,graf,list()):
                        return True
    noduri = noduri_intind_terminal(y,graf,list())
    if noduri !=False:
        for nod in noduri:
            noduri = grants(nod,graf)
            if(noduri != False):
                for nod_grant in noduri:
                    if x in noduri_intinde_terminal(nod_grant,graf,list()):
                        return True
    return False

#проверка доступа к файлу определенным образом
def check_arc(right,x,y,graf):
    for item,value in graf.items():
        if(item == x and value[0]== y):
            for i in range(1,len(value)):
                if value[i] == right: 
                    return True
    return False

#у кого есть доступ определенным образом
def is_node_with(right,y,graf):
    nodes= list()
    for item,val in graf.items():
        for value in val:
            if(value[0] == y):
                for i in range(1, len(value)):
                    if value[i]==right: nodes.append(item)
    return nodes

#может ли получить доступ
def can_share(right,x,y,graf):
    if check_arc(right,x,y,graf): return True
    s = is_node_with(right,y,graf)
    if(len(s) <= 0): return False
    noduri = noduri_intinde_terminal(s,graf,list())
    if noduri != False and x in noduri: return True
    if s in noduri_intinde_terminal(x,graf,list()): return True
    noduri_x = noduri_itinde_initial(x,graf)
    #print (noduri_x)
    if(noduri_x != False):
        noduri_s = list()
        for nod in s:
            noduri_s.extend(noduri_intinde_terminal(nod,graf,list()))
        #print(noduri_s)
        if noduri_s!=False:
            for nod_x in noduri_x:
                for nod_s in noduri_s:
                    if(is_bridge(nod_x,nod_s,graf)): return True
    
    return False


graf = stare_initiala()
noduri = list()
print_graf(graf)
print (can_share('r','S3','x',graf))

S1 -t-> S2
S1 -tg-> S3
S2 -rw-> x
S3 -r-> x
S4 -w-> y
S4 -tg-> S2
S5 -rw-> y
S5 -g-> S2
S6 -tg-> S1
S7 -g-> S3
S8 -r-> z
S8 -tg-> S1
S9 -t-> S8
S10 -tg-> S4
True
